The first step is to distribute binned weights over age, length, and sex across a defined stratum definition (we will use `"stratum_ks"`). This can be done using functions from the `get_proportions` module. The binned weight distributions are computed over values specific to aged (`dict_df_bio["specimen"]`) and unaged fish (`dict_df_bio["length"]`). This is done separately since the two datasets are processed somewhat differently before being combined in a later step.

In [ ]:
# Pre-allocate container dictionary
dict_df_weight_distr = {}

# Aged
dict_df_weight_distr["aged"] = get_proportions.binned_weights(
    length_dataset=dict_df_bio["specimen"],
    include_filter = {"sex": ["female", "male"]},
    interpolate_regression=False,
    contrast_vars="sex",
    table_cols=["stratum_ks", "sex", "age_bin"]
)

# Unaged
dict_df_weight_distr["unaged"] = get_proportions.binned_weights(
    length_dataset=dict_df_bio["length"],
    length_weight_dataset=binned_weight_table,
    include_filter = {"sex": ["female", "male"]},
    interpolate_regression=True,
    contrast_vars="sex",
    table_cols=["stratum_ks", "sex"]
)